<a href="https://colab.research.google.com/github/Shrikant727/MOMer/blob/main/speech_to_text_summary_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
#courtsey to Effortless English Page and AJ Hoge's podcast
!pip install SpeechRecognition pydub

In [30]:
import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [31]:
r=sr.Recognizer()

In [32]:
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                whole_text+=str(e)
            else:
                text = f"{text.capitalize()}. "
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [33]:
path="/content/Hitch_1_Commentary.mp3"

In [34]:
# For large audio files we are breaking them into small segments
from pydub import AudioSegment 
sound = AudioSegment.from_file(path) 
sound.export("output-file.aac", format="wav", bitrate="128k")

<_io.BufferedRandom name='output-file.aac'>

In [76]:
whole_text=get_large_audio_transcription("output-file.aac")

In [36]:
! pip install transformers

     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 895 kB 56.4 MB/s 
     |████████████████████████████████| 3.3 MB 50.7 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [37]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

In [38]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [87]:
t1=whole_text[:len(whole_text)//3]
t2=whole_text[len(whole_text)//3:2*len(whole_text)//3]
t3=whole_text[2*len(whole_text)//3:]
l=[t1,t2,t3]

In [80]:
inputs = tokenizer([t1], return_tensors='pt')

In [81]:
summary_ids = model.generate(inputs['input_ids'], max_length=500, early_stopping=False)

In [82]:
summary_ids

tensor([[    2,     0,   448, 40527,    32,    10,   372,   169,     7,  1532,
           588, 28726,  5033, 47510,     4, 27568,    33,    10, 11152,   527,
             4,   407,    47,    64,  1346,     5, 32644,     8,     5, 22810,
            11,     5, 41046,     4,   318,    89,    18,    10,   380,  5377,
            47,  1346,     5,  1086,  1068,     4,   178,   941,    77,    47,
            64,   192,    24,     4,   370,  1346,    55,     4,  1336, 32644,
            16,   341,     8,    99,  5458,     5, 28312,   873,  8244,   918,
           304,     4,   178,   253,     4,     2]])

In [83]:
txt=str([tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids])

In [84]:
txt

'["Movies are a great way to learn real conversational english. Movies have a continuous story. So you can understand the vocabulary and the phrases in the slang. If there\'s a big context you understand the whole situation. And especially when you can see it. You understand more. How vocabulary is used and what situations the vocabularies use. And end."]'

In [97]:
txt=""
for i in l:
    inputs = tokenizer([i], return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'], max_length=500, early_stopping=False)
    txt+=str([tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids])

In [98]:
txt

'["Movies are a great way to learn real conversational english. Movies have a continuous story. So you can understand the vocabulary and the phrases in the slang. If there\'s a big context you understand the whole situation. And especially when you can see it. You understand more. How vocabulary is used and what situations the vocabularies use. And end."]["If you don\'t have a teacher to help you. Like my class. I can help. But if you\'re on your own you\'re at your home you\'re doing this yourself. Do it with hitch. You put it in. And maybe watch it one time. To get the general idea but to study with it. You need to repeat. The scenes many times. It will take you some time to go through all of the movie. But the good thing is you will be learning real conversational english. As native speakers use it. On the street. In real conversation. You won\'t be learning. The textbook english. That nobody uses. Or learning. More formal english. Which people do use but. Maybe not as much. You\'ll

In [101]:
f=open('text.txt','w')
f.write(whole_text)
f.write('\n')
f.write('--------SUMMARY----------')
f.write(txt)

2217

In [106]:
with open('text.txt') as f:
    print(f.readlines())

["Hello and welcome to the commentary. For the hitch introduction. Get started. Pictures of great movie by the way it's it's really nice movie it's a romantic comedy itsfunneh starting a will smith. And. I have used hit. Many times in my classes. In san francisco. I will use the whole movie not just one scene but i will. Teach the entire movie. Little by little. And what i do it's called the focal skills movie technique. But it sounds great but but. It's really quite simple. I show one scene of a movie. 82 minutes. Maybe one minute. And i show it normal speed then i go back. Then i show it again and i pause and i explain new vocabulary slang an idiom. And i go back again. Then i this the third time i turn the sound off. And then i played part of the scene i pause and i asked the students questions about the movie. I might say. You know what's happening here describe this character. What do you think he is thinking. I why is he doing this. What is this person doing. Describe the place w